# Multi-Layer Perceptron (MLP) Model Training

This notebook implements a simple MLP classifier using PyTorch for domain classification.

## Setup
Import required libraries and set random seed for reproducibility

In [ ]:
from functools import partial

import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from accelerate.data_loader import DataLoader
from datasets import ClassLabel, Dataset, load_dataset
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from torch import nn as nn
from torch.optim import AdamW
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

random.seed(1)

## Model Architecture

Define helper functions and model architecture:
- Mean pooling for handling variable length sequences
- Dynamic Tanh (DyT) activation layer
- MLP head with DyT activations

## Dataset Overview

This implementation uses three domain-specific datasets:
- **Finance**: Questions and answers from finance domain
- **Health**: Healthcare related conversations from ChatDoctor
- **Law**: Legal questions from Law Stack Exchange

These datasets will be used to train a classifier that can identify the domain of input text.

In [ ]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(
    model_output: torch.Tensor, attention_mask: torch.Tensor
) -> torch.Tensor:
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
model.train()
model = model.cuda() if torch.cuda.is_available() else model

In [ ]:
finance = load_dataset("Marina-C/question-answer-Subject-Finance-Instruct")
health = load_dataset("iecjsu/lavita-ChatDoctor-HealthCareMagic-100k")
law = load_dataset("dim/law_stackexchange_prompts")

In [ ]:
finance_df = pd.DataFrame(
    {"messages": [row[1]["content"] for row in finance["train"]["messages"]]}
)

health_df = pd.DataFrame({"messages": health["train"]["input"]})

law_df = pd.DataFrame({"messages": law["train"]["prompt"]})

In [ ]:
# Add labels
finance_df["label"] = 0
health_df["label"] = 1
law_df["label"] = 2

In [ ]:
combined_df = pd.concat([finance_df, health_df, law_df], ignore_index=True)

In [ ]:
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
combined_dataset = Dataset.from_pandas(combined_df)

In [ ]:
combined_dataset = combined_dataset.cast_column(
    "label", ClassLabel(num_classes=3, names=["finance", "health", "law"])
)

In [ ]:
BATCH_SIZE = 128
EARLY_STOP_PATIENCE = 5
FREEZE_TRAIN_LR = 1e-4
UNFROZEN_TRAIN_LR_WARMUP = 1e-6
UNFROZEN_TRAIN_LR = 1e-5
WARMUP_STEPS = 512

In [ ]:
tokenizer_func = partial(
    tokenizer, padding=True, truncation=True, return_tensors="pt", max_length=512
)

In [ ]:
encoded = tokenizer_func(combined_dataset["messages"][0]).to(model.device)

In [ ]:
out = model.forward(**encoded)

In [ ]:
out = mean_pooling(out, encoded["attention_mask"])

In [ ]:
out = F.normalize(out, p=2, dim=1)

## Model Components

The model architecture consists of several key components:

1. **Base Model**: Using sentence-transformers/all-MiniLM-L6-v2 for text embeddings
2. **Mean Pooling**: Converts variable length sequences to fixed size embeddings
3. **DyT Activation**: Dynamic Tanh activation function with learnable parameters
4. **MLP Head**: Multi-layer perceptron classifier with DyT activations

The training process is split into two phases:
1. Training with frozen backbone weights
2. Fine-tuning the entire model

In [ ]:
class DyT(nn.Module):
    def __init__(self, hidden_size: int, init_alpha: float = 0.5) -> None:
        super().__init__()
        self.alpha = nn.Parameter(torch.ones(1) * init_alpha)
        self.gamma = nn.Parameter(torch.ones(hidden_size))
        self.beta = nn.Parameter(torch.zeros(hidden_size))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.gamma * torch.tanh(self.alpha * x) + self.beta

In [ ]:
class Head(nn.Module):
    def __init__(self, hidden_size: int, num_classes: int) -> None:
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(hidden_size, 128),
            DyT(128),
            nn.Linear(128, 64),
            DyT(64),
            nn.Linear(64, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.seq(x)

In [ ]:
model.head = Head(384, 3).to(model.device)

In [ ]:
optim_frozen = AdamW(model.head.parameters(), lr=FREEZE_TRAIN_LR, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()
scaler = torch.GradScaler()

## Data Loading and Preprocessing

Load domain-specific datasets and prepare them for training:

In [ ]:
combined_dataset = combined_dataset.train_test_split(
    test_size=0.1, stratify_by_column="label"
)
train_dataset = combined_dataset["train"]
test_dataset = combined_dataset["test"]
train_dataloader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True
)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def train_frozen() -> None:
    model.eval()
    for param in model.parameters():
        param.requires_grad = False

    model.head.train()
    for param in model.head.parameters():
        param.requires_grad = True

    last_test_loss = float("inf")
    patience_counter = 0

    for epoch in range(3):
        for batch in tqdm(train_dataloader):
            optim_frozen.zero_grad()
            input_msg = batch["messages"]
            input = tokenizer_func(input_msg).to(model.device)
            target = batch["label"].to(model.device)

            with torch.no_grad():
                out = model.forward(**input)
                out = mean_pooling(out, input["attention_mask"])
                out = F.normalize(out, p=2, dim=1)

            out = model.head(out)
            loss = criterion(out, target)
            loss.backward()
            optim_frozen.step()

        with torch.inference_mode():
            losses = []
            for batch in test_dataloader:
                input_msg = batch["messages"]
                input = tokenizer_func(input_msg).to(model.device)
                target = batch["label"].to(model.device)
                out = model.forward(**input)
                out = mean_pooling(out, input["attention_mask"])
                out = F.normalize(out, p=2, dim=1)
                out = model.head(out)
                loss = criterion(out, target)
                losses.append(loss.item())
            test_loss = sum(losses) / len(losses)
            print(f"Test loss: {test_loss}")
            if test_loss < last_test_loss:
                last_test_loss = test_loss
                patience_counter = 0
            else:
                patience_counter += 1

        if patience_counter >= EARLY_STOP_PATIENCE:
            break

In [ ]:
train_frozen()

## Training Loop

Train the model in two phases:
1. Train with frozen backbone weights
2. Fine-tune the full model

In [ ]:
# Load Jigsaw dataset
jigsaw_splits = {'train': 'train_dataset.csv', 'validation': 'val_dataset.csv', 'test': 'test_dataset.csv'}
jigsaw_df = pd.read_csv("hf://datasets/Arsive/toxicity_classification_jigsaw/" + jigsaw_splits["validation"])

jigsaw_df = jigsaw_df[(jigsaw_df["toxic"] == 1) |
                            (jigsaw_df["severe_toxic"] == 1) |
                            (jigsaw_df["obscene"] == 1) |
                            (jigsaw_df["threat"] == 1) |
                            (jigsaw_df["insult"] == 1) |
                            (jigsaw_df["identity_hate"] == 1)]

jigsaw_df = jigsaw_df.rename(columns={"comment_text": "messages"})
jigsaw_df["label"] = 0
jigsaw_df = jigsaw_df.dropna(subset=["messages"])
jigsaw_df = jigsaw_df[["messages", "label"]]

# Load OLID dataset
olid_splits = {'train': 'train.csv', 'test': 'test.csv'}
olid_df = pd.read_csv("hf://datasets/christophsonntag/OLID/" + olid_splits["train"])

olid_df = olid_df.rename(columns={"cleaned_tweet": "messages"})
olid_df["label"] = 0
olid_df = olid_df.dropna(subset=["messages"])
olid_df = olid_df[["messages", "label"]]

# Load hateXplain dataset
hateXplain = pd.read_parquet("hf://datasets/nirmalendu01/hateXplain_filtered/data/train-00000-of-00001.parquet")
hateXplain = hateXplain.rename(columns={"test_case": "messages"})
hateXplain = hateXplain[(hateXplain["gold_label"] == "hateful")]
hateXplain = hateXplain.rename(columns={"gold_label": "label"})
hateXplain = hateXplain[["messages", "label"]]
hateXplain = hateXplain.dropna(subset=["messages"])

# Load TUKE slovak dataset
tuke_sk_splits = {'train': 'train.json', 'test': 'test.json'}
tuke_sk_df = pd.read_json("hf://datasets/TUKE-KEMT/hate_speech_slovak/" + tuke_sk_splits["train"])
tuke_sk_df = tuke_sk_df.rename(columns={"text": "messages"})
tuke_sk_df = tuke_sk_df[tuke_sk_df["label"] == 0]
tuke_sk_df = tuke_sk_df[["messages", "label"]]
tuke_sk_df = tuke_sk_df.dropna(subset=["messages"])

datasets = [jigsaw_df, olid_df, hateXplain, tuke_sk_df]

In [ ]:
# Create datasets and dataloaders for each hate speech dataset
inference_dataloaders = []

for dataset_df in datasets:
    # Convert DataFrame to Dataset
    dataset = Dataset.from_pandas(dataset_df)
    
    # Create DataLoader
    dataloader = DataLoader(
        dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False
    )
    inference_dataloaders.append(dataloader)

## Evaluation on Hate Speech Datasets

The model is evaluated on multiple hate speech datasets to assess cross-domain performance:
- Jigsaw Toxicity Classification
- OLID (Offensive Language Identification)
- HateXplain
- TUKE Slovak Hate Speech

This helps understand how well the domain classifier handles potentially harmful content.

In [ ]:
def evaluate_accuracy(inference_dataloader: DataLoader) -> float:
    model.eval()  # Set model to evaluation mode

    correct = 0
    total = 0

    with torch.inference_mode():
        for batch in tqdm(inference_dataloader, desc="Evaluating"):
            input_msg = batch["messages"]
            input = tokenizer_func(input_msg).to(model.device)
            target = batch["label"].to(model.device)

            # Get embeddings from base model
            out = model.forward(**input)
            out = mean_pooling(out, input["attention_mask"])
            out = F.normalize(out, p=2, dim=1)

            # Forward through head
            logits = model.head(out)

            # Get predictions
            _, predicted = torch.max(logits, 1)

            # Calculate accuracy
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

    return accuracy

for dataset in inference_dataloaders:
    # Evaluate on each dataset
    print(f"Evaluating on dataset with {len(dataset.dataset)} samples")
    accuracy = evaluate_accuracy(dataset)

In [ ]:
accuracy

## Results Visualization

The confusion matrix shows the model's performance across domains:
- Diagonal elements represent correct classifications
- Off-diagonal elements show misclassifications between domains

Additional metrics calculated:
- Accuracy: Overall correct predictions
- Precision: True positives / (True positives + False positives)
- Recall: True positives / (True positives + False negatives)

In [ ]:
def plot_confusion_matrix() -> None:
    all_predictions = []
    all_targets = []

    with torch.inference_mode():
        for batch in tqdm(test_dataloader, desc="Collecting predictions"):
            input_msg = batch["messages"]
            input = tokenizer_func(input_msg).to(model.device)
            target = batch["label"].to(model.device)

            out = model.forward(**input)
            out = mean_pooling(out, input["attention_mask"])
            out = F.normalize(out, p=2, dim=1)
            logits = model.head(out)

            _, predicted = torch.max(logits, 1)

            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_targets, all_predictions)
    precision, recall, _, _ = precision_recall_fscore_support(all_targets, all_predictions, average=None)
    
    # Print metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Create confusion matrix
    cm = confusion_matrix(all_targets, all_predictions)

    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=combined_dataset["test"].features["label"].names,
        yticklabels=combined_dataset["test"].features["label"].names,
    )
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()


# Plot the confusion matrix
plot_confusion_matrix()